In [230]:
import json
import requests
from datetime import date
import pandas as pd
import pygal

# As specified there: https://wikimedia.org/api/rest_v1/#/
# Without your contact details you'll get an error
contact_details = "a455bcd9 on Wikipedia / @ADssx on Twitter"
headers = {
    'User-Agent': contact_details
}

# Get all projects from the Wikimedia Foundation
projects_list_url = "https://commons.wikimedia.org/w/api.php?action=sitematrix&smtype=language&format=json"
projects_list_json = requests.get(projects_list_url, headers=headers).text

projects = json.loads(projects_list_json)

ModuleNotFoundError: No module named 'pygal'

In [182]:
# Get all Wikipedias

wiki_codes = []
id = 0

# Look the whole list
for i in projects['sitematrix']:
    # Only fetch dictionnaries
    if isinstance(projects['sitematrix'][i],dict):
        # Get the language code
        code = projects['sitematrix'][i]['code']
        
        if code == 'be-x-old':
            continue

        # For each language, look at all different projects
        for j in projects['sitematrix'][i]['site']:
            # If the project is a Wikipedia, add the language code to the list of Wikipedias
            if j['code'] == 'wiki':
                wiki_codes += [code]

In [220]:
# For each Wikipedia, get the number of page views in the last month

today = date.today()

year = today.year
month = today.month - 1

# Special case for January
if today.month == 1:
    year = year - 1
    month = 12
    
last_month = date(year, month, 1)

point_in_time = last_month.strftime('%Y/%m')

# API example: https://wikimedia.org/api/rest_v1/metrics/pageviews/top-by-country/fr.wikipedia/all-access/2021/01
base_url = "https://wikimedia.org/api/rest_v1/metrics/pageviews/top-by-country/"

# For testing purposes
wiki_codes = ['en', 'fr', 'de', 'es', 'ru']

all_wikipedias = pd.DataFrame(columns=['country'])

for code in wiki_codes:
    # For each Wikipedia get the list of page views by country
    project = code + ".wikipedia"

    countries_url = base_url + project + "/all-access/" + point_in_time

    countries_json = requests.get(countries_url, headers=headers).text

    countries = json.loads(countries_json)
    
    # Transform to a dataframe
    # print(countries)
    df = pd.DataFrame.from_dict(countries['items'][0]['countries'])

    # Drop useless columns
    df = df.drop(columns=['views', 'rank'])

    # Rename one column
    df = df.rename(columns={"views_ceil": code + "_wp"})

    # Merging the dataframes on the country key with outer join
    all_wikipedias = pd.merge(all_wikipedias, df, how="outer", on="country")

# Set the country code as index
all_wikipedias = all_wikipedias.set_index("country")
all_wikipedias.head()

,en_wp,fr_wp,de_wp,es_wp,ru_wp
country,,,,,
US,3.396300e+09,19138000.0,10813000.0,48647000.0,17371000.0
GB,9.986070e+08,4339000.0,3025000.0,3017000.0,3886000.0
IN,6.783980e+08,693000.0,353000.0,343000.0,673000.0
CA,4.169020e+08,41614000.0,784000.0,1627000.0,2152000.0
AU,2.435640e+08,421000.0,372000.0,566000.0,440000.0


In [229]:
all_wikipedias.idxmax(axis=1)

country
US    en_wp
GB    en_wp
IN    en_wp
CA    en_wp
AU    en_wp
      ...  
CC    en_wp
WF    fr_wp
NU    en_wp
TK    en_wp
EH    fr_wp
Length: 243, dtype: object

In [227]:
# For each country, get the first and second top WP

# https://stackoverflow.com/a/26016605/5285608
# arank = all_wikipedias.apply(np.argsort, axis=1)
# ranked_cols = df.columns.to_series()[arank.values[:,::-1][:,:2]]
# new_frame = pd.DataFrame(ranked_cols, index=df.index)